# save the images as video
# Basler ace camera

In [4]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import matplotlib.pyplot as plt

from camcommands import camOpen

connect camera with OpenCV

In [5]:
# list cameras connected with PC

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
set_devices = tl_factory.EnumerateDevices() #enumerate devices connected

"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""

# 2. check the devices information
if len(set_devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(set_devices))
    for set_device in set_devices:
        print(set_device.GetFriendlyName()) # return readable name
        print(set_device.GetFullName()) # return unique code

total number of devices: 1
acA1300_SN01 (21566554)
Basler acA1300-30gc#00305317E75A#169.254.91.231:3956


In [41]:
# Create an array of instant cameras for the found devices and avoid exceeding a maximum number of devices.
cameras = pylon.InstantCameraArray(len(set_devices))
# Create and attach all Pylon Devices.
fps = 20 # 20 frame per sec
ExposureTime = int(1000000/fps) # in µs
Height, width = 962, 1286
for i, cam in enumerate(cameras):
    cam.Attach(tl_factory.CreateDevice(set_devices[i]))

    # Print the model name of the camera.
    print("Using device ", cam.GetDeviceInfo().GetFriendlyName())
    
    cam.Open() # open camera to get the value of the parameters and change with the new parameters
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    print(cam.Height.GetValue(), cam.Width.GetValue(), cam.ExposureTimeRaw.GetValue())
    cam.Close() # close camera after the change of parameter

Using device  acA1300_SN01 (21566554)
962 1286 50000


In [86]:
# record video and save it
for i, cam in enumerate(cameras):
    imges = pylon.PylonImage() # create empty img space
    time_initiated = datetime.now() # set the time initiated
    print("Using device ", cam.GetDeviceInfo().GetFriendlyName())

    cam.StopGrabbing()
    cam.Open() # open camera for communication

    cam.StartGrabbing() # start grabbing video or images
    video_recording_time = 1*60 #
    while cam.IsGrabbing():
        with get_writer('/Users/dkim/Desktop/basler_camera/recording/example1.mkv', macro_block_size=1) as writer:
            if i > countOfImagesToGrab:
                cam.StopGrabbing()
                cam.Close()
            else:
                writer.append_data(img.GetArray())

Using device  acA1300_SN01 (21566554)


AttributeError: no attribute 'AttachGrabResultBuffer' in GrabResult

In [29]:
# start grabbing images
cameras.StopGrabbing()
countOfImagesToGrab = 5
cameras.StartGrabbing()
for i in range(countOfImagesToGrab):
    if not cameras.IsGrabbing():
        break
    for camera in cameras:
        grabResult = camera.RetrieveResult(2000, pylon.TimeoutHandling_ThrowException)
        cameraContextValue = grabResult.GetCameraContext()
        print("Camera ", cameraContextValue, ": ", camera.GetDeviceInfo().GetModelName())
        print(datetime.now())
        img = grabResult.GetArray()
        print(img)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb_img)
cameras.StopGrabbing()


RuntimeException: Failed to open 'Basler acA1300-30gc#00305317E75A#169.254.91.231:3956'. The device is controlled by another application. Err: An attempt was made to access an address location which is currently/momentary not accessible. (0xE1018006)
 : RuntimeException thrown (file 'GigEDevice.cpp', line 280)